In [ ]:
%%bash
zcat ../../data/annotation_table_rho.txt.gz \
|grep -v "^#" \
| awk '{print $(NF-0)}' \
|sort -n - \
|uniq -c >rho_counts.txt

In [ ]:
import os
os.chdir('../../data/diversity/')

In [ ]:
import pandas as pd
rho_df = pd.read_table('rho_counts.txt', sep="\s", na_values=".", names=['count', 'rho'])
rho_df.head()

print(rho_df.rho.min(), rho_df.rho.max())



based on figure 1 the vast majority of windows are done by 0.06

so lets go 50 windows from 0 to 0.06

In [ ]:
bins = [(i/1000, i/1000 + 1/1000) for i in range(61)]

r_hist  = {}
for b in bins:
    r_hist[b] = {'sites':0, 'COs':0}
    

for l in open('rho_counts.txt'):
    try:
        site_count, rho = int(l.strip().split()[0]), float(l.strip().split()[1])
    except:
        print(l)
        continue
    for b in bins:
        if  b[0] <= rho < b[1]:
            r_hist[b]['sites'] += site_count
            break

            

In [ ]:
for l in open("all_break_points_rho.txt"):
    if not l.startswith('cross'):
        rho = float(l.strip().split(',')[-1])
        for b in bins:
            if  b[0] <= rho < b[1]:
                r_hist[b]['COs'] += 1
                break

In [ ]:
r_hist

In [ ]:

import matplotlib.pyplot
import pylab
import numpy as np 

mid_x = [(b[1] + b[0])/2 for b in bins if r_hist[b]['sites'] >0]
CO_density =  [r_hist[b]['COs']/r_hist[b]['sites'] for b in bins if r_hist[b]['sites'] >0 ]

CO_density

matplotlib.pyplot.scatter(mid_x,np.log(CO_density))


In [ ]:
rho_midpoints = [(b[0]+b[1])/2 for b in bins ]
COs = [r_hist[b]['COs'] for b in bins]
CO_density =  [r_hist[b]['COs']/r_hist[b]['sites'] for b in bins]
sites = [r_hist[b]['sites'] for b in bins]


df = pd.DataFrame.from_dict({'rho_midpoints':rho_midpoints,
                             "COs": COs,
                             "CO_density":CO_density,
                             "sites":sites,
                             "bins":bins
                            })

df['log_CO_density'] = np.log(df.CO_density+1)

In [ ]:
df

In [ ]:
df.plot(kind='scatter', x='rho_midpoints', y='log_CO_density', ylim=(0, 0.0001) )

In [ ]:
import scipy.stats  as stats

#df_filter = df[(df.sites>500000)]
df_filter = df[df.COs >0]
df_filter = df
df_filter.plot(kind='scatter',x='rho_midpoints',y='log_CO_density')
print("Correlation: ", df_filter.rho_midpoints.corr(df_filter.CO_density))
fit  = stats.pearsonr(df_filter['rho_midpoints'], df_filter['CO_density'])
print(fit)
#df_filter

In [ ]:
df.to_csv("rho_to_CO_density.txt")

In [ ]:
df.dropna()